In [4]:
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 216kB/s]
100%|██████████| 200k/200k [00:00<00:00, 216kB/s]


In [6]:
import zipfile
zipref = zipfile.ZipFile('orders.csv.zip')
zipref.extractall()
zipref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='orders.csv.zip' mode='r'>>

In [7]:
import pandas as pd
df = pd.read_csv('orders.csv')
df.head(20)

df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

# Rename column names (Lowercase and separated by '_')
df.columns = df.columns.str.lower().str.replace(' ','_')
df.columns
df.head()

# Create new column for Discount price, Sale price and Profit
df['discount_price'] = df['list_price'] * df['discount_percent']*.01
df['sale_price'] = df['list_price'] - df['discount_price']
df['profit'] = df['sale_price'] - df['cost_price']
df

# convert order date from object data type to datetime
df['order_date'] = pd.to_datetime(df['order_date'],format="%Y-%m-%d")
df['order_date']

# Drop cost price, list price and discount percent columns
df.drop(columns=['cost_price','list_price','discount_percent'],inplace=True)
df

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount_price,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,3,1.2,28.8,-1.2
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,2,3.6,86.4,16.4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,2,5.2,254.8,34.8
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,4,0.9,29.1,-0.9


In [2]:
# Load the data to SQL Server
import mysql.connector
from mysql.connector import Error

def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',  # or 'localhost'
            user='root',
            password='admin123',  # replace with your actual password
            database='orders'  # replace with your actual database name
        )
        if connection.is_connected():
            print("Connected to MySQL server")
    except Error as e:
        print(f"Error: {e}")
    return connection

# Establish connection
connection = create_connection()

# Check connection
if connection:
    print("Connection established successfully!")
    # Close the connection
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")
else:
    print("Failed to establish connection.")


Connected to MySQL server
Connection established successfully!
MySQL connection is closed


In [8]:
import pandas as pd
from sqlalchemy import create_engine

# Define your MySQL connection details
user = 'root'
password = 'admin123'
host = '127.0.0.1'
database = 'orders'

# Create an SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Write the DataFrame to a SQL table
df.to_sql('df_orders', con=engine, index=False, if_exists='append')
print("DataFrame written to table `df_orders` successfully.")

DataFrame written to table `df_orders` successfully.


In [13]:
df.to_sql('df_orders', con=engine, index=False, if_exists='append')
print("DataFrame written to table `df_orders` successfully.")

DataFrame written to table `df_orders` successfully.
